# DKN : Deep Knowledge-Aware Network for News Recommendation
DKN은 더 좋은 뉴스 추천을 위해 지식 그래프의 정보를 통합하는 딥러닝 모델이다. 특히, DKN은 지식 그래프 표현 학습의 방법으로 *TransX*를 사용하고, *KCNN*이라 불리는 CNN 프레임워크를 사용한다. 이를 통해 엔티티 임베딩과 단어 임베딩을 결합하고 뉴스 기사에 대한 최종 임베딩 벡터를 생성한다. attention-based neural scorer를 통해 CTR(Click-Through_Rate) 예측을 만든다.   

## Properties of DKN
- DKN은 전통적인 ID 기반의 협업 필터링보다는 CTR 예측을 위한 content-based 딥 모델이다.
- It makes use of knowledge entities and common sense in news content via joint learning from semantic-level and knowledge-level representations of news articles.
- DKN은 attention 모듈을 이용하여 사용자의 집계된 과거 표현을 동적으로 계산한다.

## Data format
- training / validation / test files : The format is:   
`[label] [userid] [CandidateNews]%[impressionid]`   
e.g. `1 train_U1 N1%0`   
이 파일들의 각 줄은 한 인스턴스를 대변한다. `Impressionid`는 impression 세션에서 퍼포먼스를 평가하기 위해 사용된다. 평가 중에만 사용되며, 훈련 데이터에 대해서는 0으로 세팅한다.   
- user history file : The format is:   
`[Userid] [newsid1, newsid2, ...]`   
이 파일의 각 줄은 유저의 클릭 히스토리를 대변한다. config 파일에서 `history_size` 파라미터 : 유저의 클릭 히스토리로 사용할 최대치를 설정해야 한다. 만약 유저의 클릭 히스토리가 파라미터보다 크면 마지막 수를 자동적으로 유지하고, 클릭 히스토리가 파라미터보다 작으면 자동적으로 0을 패딩한다.   
- document feature file : The format is:   
`[Newsid] [w1, w2, ..., wk] [e1, e2, ..., ek]`   
이 파일엔 뉴스 기사들의 단어와 엔티티 특성이 들어있다.
- word embedding / entity embedding / context embedding files:
사전학습된 임베딩들의 `*.npy` 파일들이다. 각 파일은 [n+1, k]의 2차원 행렬이고, n은 해쉬 딕셔너리의 단어들(엔티티들)의 수이고 k는 임베딩의 차원의 수이다.

In this experiment, we used GloVe[4] vectors to initialize the word embedding. We trained entity embedding using TransE[2] on knowledge graph and context embedding is the average of the entity's neighbors in the knowledge graph.

## MIND dataset
이 노트에선 MIND 데이터셋의 일부분-*MIND demo*-만 사용합니다. 5000명의 유저, 9432건의 뉴스기사와 6134개의 imporession logs를 다룹니다.

## Global Settings and imports

In [1]:
import sys
import os
import scrapbook as sb
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from tempfile import TemporaryDirectory
from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources, prepare_hparams
from recommenders.models.deeprec.models.dkn import DKN
from recommenders.models.deeprec.io.dkn_iterator import DKNTextIterator

print('System version: {}'.format(sys.version))
print('Tensorflow version: {}'.format(tf.__version__))


System version: 3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]
Tensorflow version: 2.7.3


## Download and load data

In [2]:
tmpdir = TemporaryDirectory()
data_path = os.path.join(tmpdir.name, 'mind-demo-dkn')

yaml_file = os.path.join(data_path, r'dkn.yaml')
train_file = os.path.join(data_path, r'train_mind_demo.txt')
valid_file = os.path.join(data_path, r'valid_mind_demo.txt')
test_file = os.path.join(data_path, r'test_mind_demo.txt')

news_feature_file = os.path.join(data_path, r'doc_feature.txt')
user_history_file = os.path.join(data_path, r'user_history.txt')

wordemb_file = os.path.join(data_path, r'word_embeddings_100.npy')
entityemb_file = os.path.join(data_path, r'TransE_entity2vec_100.npy')
contextemb_file = os.path.join(data_path, r'TransE_context2vec_100.npy')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/deeprec/', tmpdir.name, 'mind-demo-dkn.zip')

100%|█████████████████████████████████████| 11.3k/11.3k [00:06<00:00, 1.86kKB/s]


In [3]:
tmpdir

<TemporaryDirectory '/tmp/tmp6vr48dhj'>

## Create Hyper-parameters

In [13]:
epochs = 10
history_size = 50
batch_size = 64

hparams = prepare_hparams(yaml_file, news_feature_file=news_feature_file,
                         user_history_file=user_history_file,
                         wordEmb_file=wordemb_file,
                         entityEmb_file=entityemb_file,
                         contextEmb_file=contextemb_file,
                         epochs=epochs,
                         history_size=history_size,
                         batch_size=batch_size)
print(hparams)

HParams object with values {'use_entity': True, 'use_context': True, 'cross_activation': 'identity', 'user_dropout': False, 'dropout': [0.0], 'attention_dropout': 0.0, 'load_saved_model': False, 'fast_CIN_d': 0, 'use_Linear_part': False, 'use_FM_part': False, 'use_CIN_part': False, 'use_DNN_part': False, 'init_method': 'uniform', 'init_value': 0.1, 'embed_l2': 1e-06, 'embed_l1': 0.0, 'layer_l2': 1e-06, 'layer_l1': 0.0, 'cross_l2': 0.0, 'cross_l1': 0.0, 'reg_kg': 0.0, 'learning_rate': 0.0005, 'lr_rs': 1, 'lr_kg': 0.5, 'kg_training_interval': 5, 'max_grad_norm': 2, 'is_clip_norm': 0, 'dtype': 32, 'optimizer': 'adam', 'epochs': 10, 'batch_size': 64, 'enable_BN': True, 'show_step': 10000, 'save_model': False, 'save_epoch': 2, 'write_tfevents': False, 'train_num_ngs': 4, 'need_sample': True, 'embedding_dropout': 0.0, 'EARLY_STOP': 100, 'min_seq_length': 1, 'slots': 5, 'cell': 'SUM', 'doc_size': 10, 'history_size': 50, 'word_size': 12600, 'entity_size': 3987, 'data_format': 'dkn', 'metrics':

## Train the model

In [14]:
model = DKN(hparams, DKNTextIterator)

/home/qwer/anaconda3/envs/recommender3713/lib/python3.7/site-packages/recommenders/models/deeprec/models/dkn.py:309: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  training=self.is_train_stage,
/home/qwer/anaconda3/envs/recommender3713/lib/python3.7/site-packages/keras/legacy_tf_layers/normalization.py:455: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)
/home/qwer/anaconda3/envs/recommender3713/lib/python3.7/site-packages/recommenders/models/deeprec/models/dkn.py:197: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNorm

In [8]:
os.path.getsize(valid_file)

389962

In [ ]:
model.run_eval(valid_file)

In [6]:
model.fit(train_file, valid_file)

KeyboardInterrupt: 

## Evaluate the DKN model

In [12]:
res = model.run_eval(test_file)
print(res)

{'auc': 0.5968, 'group_auc': 0.5654, 'mean_mrr': 0.1816, 'ndcg@5': 0.1819, 'ndcg@10': 0.244}


fit 후 성능이 오르긴 올랐다?

In [13]:
sb.glue('res', res)